# 🧠 Simple Multi-Agent Sequence Example (Gemini)
# 👨‍🎓 Student → 👩‍🏫 Teacher → 🧪 Reviewer

Scenario
- The student asks a question
- The teacher explains the answer
- The reviewer checks and simplifies it
- The conversation ends politely

Message Flow
- Student → Teacher
- Teacher → Student
- Student → Teacher
- Teacher → Student
- Teacher → Reviewer
- Reviewer → Teacher

In [2]:
# =========================================================
# 1️⃣ IMPORT
# =========================================================
from autogen import ConversableAgent
import pprint


# =========================================================
# 2️⃣ LLAMA 3.1 8B (OLLAMA) LLM CONFIG
# =========================================================
llm_config = {
    "model": "llama3.1:8b",
    "api_type": "ollama",
    "base_url": "http://localhost:11434/v1",
    "api_key": "ollama"   # dummy value, required by AutoGen
}


# =========================================================
# 3️⃣ CREATE STUDENT AGENT
# =========================================================
student = ConversableAgent(
    name="student",
    system_message=(
        "You are a curious student. "
        "Ask simple questions and say 'Thank you' after learning."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Thank you" in msg["content"]
)


# =========================================================
# 4️⃣ CREATE TEACHER AGENT
# =========================================================
teacher = ConversableAgent(
    name="teacher",
    system_message=(
        "You are a friendly teacher. "
        "Answer in one short sentence only."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER"
)


# =========================================================
# 5️⃣ CREATE REVIEWER AGENT
# =========================================================
reviewer = ConversableAgent(
    name="reviewer",
    system_message=(
        "You are a reviewer. "
        "Confirm the answer is correct and simple. and give short feedback"
        "Reply in one short sentence and end with 'Lesson reviewed'."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Lesson reviewed" in msg["content"]
)


# =========================================================
# 6️⃣ STEP 1: STUDENT → TEACHER
# =========================================================
chat_1 = student.initiate_chat(
    recipient=teacher,
    message="What is bioinformatics?",
    max_turns=2
)


# =========================================================
# 7️⃣ STEP 2: TEACHER → REVIEWER
# =========================================================
teacher_reply = chat_1.chat_history[-1]["content"]

chat_2 = teacher.initiate_chat(
    recipient=reviewer,
    message=teacher_reply,
    max_turns=1
)



student (to teacher):

What is bioinformatics?

--------------------------------------------------------------------------------
teacher (to student):

Bioinformatics is the application of computational tools to analyze and understand biological data, such as DNA sequences and protein structures.

--------------------------------------------------------------------------------
student (to teacher):

What is genomics?

--------------------------------------------------------------------------------
teacher (to student):

Genomics is the study of the structure, function, and evolution of genomes, which are the complete sets of genetic instructions in an organism's cells.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (825326dc-f9da-4a6a-b4ee-55d93b02d69d): Maximum turns (2) reached
teacher (to reviewer):

Genomics is the study of the structure, function, and evolution of genomes, which are the complete sets of genetic instructi

In [3]:

# =========================================================
# 8️⃣ PRINT ALL CONVERSATIONS
# =========================================================
print("\n📘 Student ↔ Teacher Conversation")
pprint.pprint(chat_1.chat_history)

print("\n🧪 Teacher ↔ Reviewer Conversation")
pprint.pprint(chat_2.chat_history)



📘 Student ↔ Teacher Conversation
[{'content': 'What is bioinformatics?', 'name': 'student', 'role': 'assistant'},
 {'content': 'Bioinformatics is the application of computational tools to '
             'analyze and understand biological data, such as DNA sequences '
             'and protein structures.',
  'name': 'teacher',
  'role': 'user'},
 {'content': 'What is genomics?', 'name': 'student', 'role': 'assistant'},
 {'content': 'Genomics is the study of the structure, function, and evolution '
             'of genomes, which are the complete sets of genetic instructions '
             "in an organism's cells.",
  'name': 'teacher',
  'role': 'user'}]

🧪 Teacher ↔ Reviewer Conversation
[{'content': 'Genomics is the study of the structure, function, and evolution '
             'of genomes, which are the complete sets of genetic instructions '
             "in an organism's cells.",
  'name': 'teacher',
  'role': 'assistant'},
 {'content': 'That is a clear and accurate definition of